# First couple of dates with Julia...

Hello, so I have spent the past week or so getting familiar with Julia. This is a really neat language! It is being heavily developed by [MIT](http://news.mit.edu/2018/mit-developed-julia-programming-language-debuts-juliacon-0827) and is already finding some very neat real world use cases at places like [FAA](https://juliacomputing.com/case-studies/lincoln-labs.html), [Federal Reserve](https://juliacomputing.com/case-studies/ny-fed.html), Google, Amazon... Julia was designed as a modern programming language that is capable of solving the two language problem that many people face. I kind of liken Julia to a designer test tube baby spawn of Python, R, and C. It has the ease of development of Python, the pure math pedigree of R and the performance of C. 

Julia is not a pure object oriented programing language (OOP), rather it uses what is called a [Duck Type](https://en.wikipedia.org/wiki/Duck_typing) language. This is the first time I have ever heard of it. So, I made this notebook mostly to help myself attempt to understand and learn about the differences between Python's Class based style of doing OOP and Julia's Type based pseudo OOP style. Hopefully my code examples and ramblings can be of some use to others who have stumbled on Julia. [Type-Dispatch Design: Post Object-Oriented Programming for Julia](http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/) was a good article that helped me a lot. I also spent a lot of time reading the [Julia documentation](https://docs.julialang.org/en/v1/) as it is very detailed and well written. 

## Intro to Types

In [1]:
@time a = 1:1000000 # this happens at )(1) time! its not really creating an array but is rather just a function.

  0.000003 seconds (7 allocations: 288 bytes)


1:1000000

In [2]:
a[94] # we can index `a` 

94

In [3]:
@time b = collect(1:10000000)
println("\n\nWe can index `a` or `b` the same, but a was created in a fraction of the time.")
println("a[69] = ", a[69])
println("b[69] = ", b[69])

  0.030751 seconds (30 allocations: 76.297 MiB, 27.98% gc time)


We can index `a` or `b` the same, but a was created in a fraction of the time.
a[69] = 69
b[69] = 69


Just out of curiosity lets see how long it would take to create an array of equal size in Python. 

In [1]:
import numpy as np
from datetime import datetime

t1 = datetime.now()
c = np.arange(10000000)
t2 = datetime.now()
print((t2-t1).total_seconds(), "seconds")
print("\n\nAnd for unitfority sake lets index `c`.")
print("c[69] = ", c[69])

0.024788 seconds


And for unitfority sake lets index `c`.
c[69] =  69


In [4]:
typeof(a) == typeof(b)

false

In [5]:
typeof(a), typeof(b)

(UnitRange{Int64}, Array{Int64,1})

In [6]:
typeof(a) <: AbstractArray

true

In [7]:
typeof(b) <: AbstractArray

true

Looks like both Julia and Python can populate an array about the same speed(I ran it several times and it was a toss up). But the real takeaway here is that `a` and `b` are not of the exact same Type. If we inspect the two variables we will see that `a` is a `UnitRange` type and `b` is an `Array` of Int64s but they are both of the `AbstractArray` type. 

Julia has the ability to have a hierarchy of Types. This is not a new concept as other languages I think have this, with both `Float64` and `Int64` being of `Number` type, but what is interesting here (at least for me) is that we can use this as a pseudo OOP.  I wrote this as I was learning and am now revising it after I have figured a few additional things out, but basically all this is made possible by the languages ability to do multiple dispatches. What does this mean? Because, I didn't know what it meant, not being a real `Computer Scientist`. So my naive mind basically boils this down to being able to smartly choose the correct `Type` based on the value given. So if I declare a value `X` and set it equal to `10`, the interpreter is intelligent enough to inspect `10` and understand it is an `Int` and not a `String` or `Float`. This [article](http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/) helped me understand things and shows how the code gets compiled.

I really know Python and so I am going to compare Julia to Python. So what is a class in Python or other OOP language but really just a store of attributes (data) about said object. The object then has methods that can be used to get the data or do some type of programming magic related to that object. Objects in Python and other OOP languages can also inherit attributes from parents saving lines of code, and making them easier to maintain. With this in mind I set out to attempt to do some similar type of things in Julia. 

### Declare abstract types

First thing we need to do is declare our abstract types and their hierarchy. The defining I am doing here could be done across multiple files or in different parts of your code if you are working on a larger project but for demonstrative purposes lets just do it all right here. 

In [8]:
abstract type AbstractPerson end
# NewType <: ParentType
abstract type AbstractCivilian <: AbstractPerson end
abstract type AbstractMilitary <: AbstractPerson end

# I want to be able to alter attributes so I am making it mutable, by default its not..
mutable struct Person <: AbstractPerson
    fname::String
    lname::String
    sex::String
    # you don't have to assign a specific type... its just as fast if you don't
    age
    education
end

mutable struct Civilian <: AbstractCivilian
    #=
    this is where the fake inheritence comes into play. 
    I haven't found a community wide design pattern yet but plan on adopting `baseType` as my
    default inheritence object name. This allows me to always reference the same thing
    in order to gain access to inherited attributes
    =#
    baseType::Person
    profession::String
    job::String
end

mutable struct Military <: AbstractMilitary
    baseType::Person
    branch::String
    mos::String
    rank::String
end



### Instantiating Types

To instantiate one of these types is very similar to Python, kind of... If you want to use named attributes to init the type, you need to define a constructor method/function. 

In [9]:
# without making a constructor you just pass in your attributes in order
john_doe = Person("John", "Doe", "Male", nothing, nothing)

#But I think this could get confusing and/or we want to be able to have some default values
# This is really just a function/method condensed down to a single line
Person(;fname="John",lname="Doe",sex="M",age="Unknown",education="Unknown") = Person(fname,lname,sex,age,education)

# This is the long form of the above. Both ways work, this is probably easier to understand though
function Civilian(; fname="John",
                    lname="Doe",
                    sex="M",
                    age="Unknown",
                    education="Unknown",
                    profession="Unknown",
                    job="Unknown")
    baseType = Person(fname,lname,sex,age,education)
    return Civilian(baseType, profession,job)
end

function Military(; fname="John",
                    lname="Doe",
                    sex="M",
                    age="Unknown",
                    education="Unknown",
                    branch="Unknown",
                    mos="None",
                    rank="Boot")
    baseType = Person(fname,lname,sex,age,education)
    return Military(baseType,branch,mos,rank)
end

Military

In [10]:
println("Attributes of Person:")
println(fieldnames(Person))

println("Attributes of Military:")
println(fieldnames(Military))

println("Attributes of Civilian:")
println(fieldnames(Civilian))

Attributes of Person:
(:fname, :lname, :sex, :age, :education)
Attributes of Military:
(:baseType, :branch, :mos, :rank)
Attributes of Civilian:
(:baseType, :profession, :job)


In [11]:
matt = Military(
    fname="Matt",
    lname="Camp",
    age=33,
    education="Masters",
    branch="Marine Corps",
    mos="0811/0231",
    rank="Sgt."    
)

candace = Civilian(
    fname="Cadace",
    lname="Camp",
    sex="F",
    age=31,
    profession="Domestic Engineer",
    job="Dog Mom"
)
#We can access the attributes of the Type just like we can the attributes of a Class in Python.
println("This is " * matt.baseType.fname * " " * matt.baseType.lname *" and he was in the " * matt.branch * ".")

This is Matt Camp and he was in the Marine Corps.


### Create Methods/Functions

Having to join my first and last name together like I did above kind of is a pain. Lets create a method to return my full name, and while we are at it lets create a couple other methods. One thing to take note of here is that because Julia allows multiple dispatches we can create the same function/method multiple times for the different Types that we want it to be used on. 

In [12]:
get_fullname(x::AbstractPerson) = x.fname * " " * x.lname
# we can either do this which isn't too bad
get_fullname(matt.baseType)

"Matt Camp"

One thing to take note of is that I couldn't just pass in the variable `matt`. I had to do `matt.baseType`. This is one of the limitation of faking inheritence. Because of the multiple dispatching ability of Julia we can do the following but it means writing more code, so its not perfect. 

In [13]:
# or if we want to make things easier on the front end we can do this
# I don't think this is perfect but its not horrible.
get_fullname(x::AbstractCivilian) = x.baseType.fname * " " * x.baseType.lname
get_fullname(x::AbstractMilitary) = x.baseType.fname * " " * x.baseType.lname

get_fullname(candace)

"Cadace Camp"

In [14]:
function get_profession_type(x::AbstractMilitary)
    if x.branch == "Coast Guard"
        return "Civilian" #;-P
    else
        return "Military"
    end
end

get_profession_type(x::AbstractCivilian) = "Civilian"

get_profession(x::AbstractMilitary) = x.mos
get_profession(x::AbstractCivilian) = x.profession

function get_title(x::AbstractPerson)
    if x.sex == "M"
        return "Mr."
    else
        return "Ms."
    end
end

function get_title(x::AbstractCivilian)
    if x.baseType.sex == "M"
        return "Mr."
    else
        return "Ms."
    end
end

get_title(x::AbstractMilitary) = x.rank

function get_job_title(x::AbstractMilitary)
    if x.branch == "Marine Corps"
        return "Marine"
    elseif x.branch == "Army"
        return "Soldier"
    elseif x.branch == "Navy"
        return "Sailor"
    elseif x.branch == "Air Force"
        return "Airman"
    else
        if x.basetType.sex == "M"
            return "Mr."
        else
            return "Ms."
        end
    end
end

get_job_title (generic function with 1 method)

Creating all these different methods seems to me like it could get messy and a uniform grouping method might be needed to keep things clean and organized. As I see it there are two ways to organize these methods. They can be organized by function or by Type. Coming from Python I think organizing the methods by the Type they support would be the most logical, though I could see a reason to group them by function. 


### Converting Types

One intersting thing I found in my research and learning of Julia, is that you could convert a Type to another Type. This could be useful or not, I don't know but I thought the following was an intersting use case.

Also I got an idea in the back of my head on how you could do inheritence this way too and it would avoid the nested type structure. But it might end up being more code and could get messy. Just a thought to anyone willing to play around with it. 

In [15]:
function enlist(::Type{AbstractMilitary}, c::AbstractCivilian) 
    return Military(fname=c.baseType.fname, 
                    lname=c.baseType.lname, 
                    age=c.baseType.age,
                    sex=c.baseType.sex,
                    education=c.baseType.education, 
                    branch="Unknown", 
                    mos="No MOS", 
                    rank="Rct." )
end

function eas(::Type{AbstractCivilian}, m::AbstractMilitary)
    return Civilian(fname=m.baseType.fname, 
                    lname=m.baseType.lname, 
                    age=m.baseType.age, 
                    sex=m.baseType.sex,
                    education=m.baseType.education,
                    profession="Disabled Veteran",
                    job="Unemployed" )
end

Military(c::AbstractCivilian) = enlist(AbstractMilitary, c)
Civilian(m::AbstractMilitary) = eas(AbstractCivilian, m)

matt = Civilian(matt)
matt.job = "Artificial Intelligence Software Developer"
matt.profession = "Data Science"
candace = Military(candace)

matt

Civilian(Person("Matt", "Camp", "M", 33, "Masters"), "Data Science", "Artificial Intelligence Software Developer")

Till now I've been using method and function interchangeably. When looking at this function called `methods()` and looking more closely at the [Julia documentation](https://docs.julialang.org/en/v1/manual/methods/) I think I finally figured it out in my head. Julia has functions, and most things are going to be a function. Functions accept arguments in the form tuples containing different `Types`. These types are constructed via Methods. Each Type can have multiple Methods for construction. So if we look at the following we will see that each Type has multiple Methods that I defined previously. 

So to rephrase this incase my logic is hard to follow. A Type is the Object that contains information. To make a Type you can have multiple Methods. Where some Methods take in named attributes and others convert some Type to another Type. Functions are for lack of a better word, the program. They are the things that act upon Types. 

Sorry for all the rambling but I am learning this stuff too. I hope this helps someone else wrap their head around the subject. 

In [16]:
methods(Person)

# 3 methods for generic function "(::Type)":
[1] Person(; fname, lname, sex, age, education) in Main at In[9]:6
[2] Person(fname::String, lname::String, sex::String, age, education) in Main at In[8]:8
[3] Person(fname, lname, sex, age, education) in Main at In[8]:8

In [17]:
methods(Military)

# 4 methods for generic function "(::Type)":
[1] Military(; fname, lname, sex, age, education, branch, mos, rank) in Main at In[9]:28
[2] Military(baseType::Person, branch::String, mos::String, rank::String) in Main at In[8]:29
[3] Military(c::AbstractCivilian) in Main at In[15]:22
[4] Military(baseType, branch, mos, rank) in Main at In[8]:29

In [18]:
methods(Civilian)

# 4 methods for generic function "(::Type)":
[1] Civilian(; fname, lname, sex, age, education, profession, job) in Main at In[9]:16
[2] Civilian(baseType::Person, profession::String, job::String) in Main at In[8]:23
[3] Civilian(m::AbstractMilitary) in Main at In[15]:23
[4] Civilian(baseType, profession, job) in Main at In[8]:23

### Conclusion

So, I just want to surmise that I think Julia to be a really neat language that could potentially surpass Python in popularity. It is both faster and it can import Python packages directly and vice versa  thanks to [PyJulia](https://github.com/JuliaPy/pyjulia).

#### P.S.

Ok, so I was already to wrap this up but I read more and found a package that does some metaprogramming to do inheritance. I found a [package](https://github.com/tbreloff/ConcreteAbstractions.jl) that will basically allow you to do inheritance in a more natural way. But I have yet to get it to work with Julia v1.0. I am not sure if this is due to my ignorance or if it is an incompatibility problem since the package hasn't had an update since 2016. 